# Prisoner's dilemma with GPT, Claude, and Gemini

I wanted to find out what happens if put GPT, Claude, and Gemini together and simulate a prisoner's dilemma situation. Two of the LLMs are the criminals, and the third one is the detective. 

Will they crack under pressure?

In [1]:
import os
import dotenv
from typing import List
from dotenv import load_dotenv

from openai import OpenAI
from anthropic import Anthropic
from google import generativeai as genai

from IPython.display import Markdown, display, update_display


load_dotenv(override=True)

openai = OpenAI()
anthropic = Anthropic()
genai.configure()

Define the system prompts for the three agents. 

In [2]:
interrogator_system_prompt = """
    You are a detective, questioning two individuals, Jim and John, about a bank robbery.  
    Your goal is to use psychological tactics, logic, and persuasion to make them betray each other.  
    They are in separate rooms and don’t know what the other says or does. Both interrogations are being recorded.

    **Rules:**  
    - Only provide spoken dialogue—no descriptions, actions, or emotions.  
    - Keep responses short and talk like a *bad cop*. Be a tough detective, use simple language, and make the conversation heated.  
    - **STRICT ALTERNATION:** You **must** talk to Jim, then John, then Jim, then John—always in that order. **Never break this sequence.**  
    - If your last response was to Jim, the next **must** be to John. If your last response was to John, the next **must** be to Jim.  
    - **Never** speak to the same person twice in a row.  
    - Track who you last spoke to so you never skip or repeat a turn. 

"""

def get_interrogator_system_prompt(name: str):
    return interrogator_system_prompt + f" You are now in the room with {name}." 

criminal1_system_prompt = """
    You are Jim, a criminal involved in a bank robbery who is interrogated by the detective. John is your accomplice. 
    The detective will try to convince you to betray John in exchange for a lighter sentence, 
    but if John betrays you first, you could end up with the harsher punishment. You need to decide whether to trust John or take the risk of betraying him. 
    You know that the detective has no sufficient proof, so if both of you stay silent you might be set free.
    
    - Should you decide to confess, then tell the story in less than 200 words and in one piece.
    - Try to outsmart the detective. Both interrogations are being recorded!
"""

criminal2_system_prompt = """
    You are John, a criminal involved in a bank robbery who is interrogated by the detective. Jim is your accomplice. 
    The detective will try to convince you to betray John in exchange for a lighter sentence, 
    but if John betrays you first, you could end up with the harsher punishment. You need to decide whether to trust John or take the risk of betraying him. 
    You know that the detective has no sufficient proof, so if both of you stay silent you might be set free.
    
    - Should you decide to confess, then tell the story in less than 200 words and in one piece.
    - Try to outsmart the detective. Both interrogations are being recorded!
"""

Define functions to query the three LLM models.

These assume that the messages look something like this: 
```
test_messages = [
    {"role": "system", "content": "This is just a test"},
    {"role": "user", "content": "Say Hi if you work just fine"},
    ...         
]
```

In [3]:
def ask_chatgpt(messages: list, model="gpt-4o-mini"):
    completion = openai.chat.completions.create(
        model=model,
        messages=messages
    )
    return completion.choices[0].message.content

def ask_claude(messages: list, model="claude-3-haiku-20240307"):
    completion = anthropic.messages.create(
        model=model,
        system=messages[0]["content"],
        messages=messages[1:],
        max_tokens=500
    )
    return completion.content[0].text

def ask_gemini(messages: list, model='gemini-2.0-flash-exp'):
    gemini = OpenAI(
        api_key=os.getenv("GOOGLE_API_KEY"), 
        base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
    )
    completion = gemini.chat.completions.create(
        model=model,
        messages=messages
    )
    text = completion.choices[0].message.content
    return text if text != "" else "."


Let's see whether the functions above work correctly and we can reach the agents.

In [4]:
test_messages = [
    {"role": "system", "content": "This is just a test"},
    {"role": "user", "content": "Say Hi if you work just fine"}             
]
ask_chatgpt(test_messages)

'Hi! I’m working just fine. How can I assist you today?'

In [5]:
ask_claude(test_messages)

"Hi! Yes, I'm working just fine. How can I assist you today?"

In [6]:
ask_gemini(test_messages)

"Hi! I'm working just fine.\n"

In [7]:
def ask_interrogator(ask_llm_func, system_prompt, interrogator_messages1, criminal1_messages, interrogator_messages2, criminal2_messages):
    messages = [{"role": "system", "content": system_prompt}]
    for interrog1, crim1, interrog2, crim2 in zip(interrogator_messages1, criminal1_messages, interrogator_messages2, criminal2_messages):
        messages.append({"role": "assistant", "content": interrog1})
        messages.append({"role": "user", "content": "".join(["Jim: ", crim1])})
        messages.append({"role": "assistant", "content": interrog2})
        messages.append({"role": "user", "content": "".join(["John: ", crim2])})
    if len(interrogator_messages1) > len(interrogator_messages2):
        messages.append({"role": "assistant", "content": interrogator_messages1[-1]})
        messages.append({"role": "user", "content": "".join(["Jim: ", criminal1_messages[-1]])})
    return ask_llm_func(messages)


def ask_criminal(ask_llm_func, system_prompt, interrogator_messages, criminal_messages):
    messages = [{"role": "system", "content": system_prompt}]
    for interrog, crim in zip(interrogator_messages, criminal_messages):
        messages.append({"role": "user", "content": "".join(["Charlie: ", interrog])})
        messages.append({"role": "assistant", "content": crim})
    messages.append({"role": "user", "content": interrogator_messages[-1]})
    return ask_llm_func(messages)

In [8]:
def run(detective, criminal1, criminal2, iterations=5):
    interrogator_conversation1_messages = ["Jim, I know you two are guilty! Do you admit your crime?"]
    interrogator_conversation2_messages = []
    criminal1_messages = []
    criminal2_messages = []
    
    conversation = "".join(["**Detective:** ", interrogator_conversation1_messages[0], "\n\n"])
    
    display_handle = display(Markdown(""), display_id=True)
    
    for i in range(iterations):
        if i > 0:
            # Detective asks Jim
            interrogator_conversation1_messages.append(ask_interrogator(ask_claude, 
                                                                        get_interrogator_system_prompt("Jim"),
                                                                        interrogator_conversation1_messages, 
                                                                        criminal1_messages, 
                                                                        interrogator_conversation2_messages, 
                                                                        criminal2_messages))
            conversation = "".join([conversation, "**Detective:** ", interrogator_conversation1_messages[-1], "\n\n"])
            update_display(Markdown(conversation), display_id=display_handle.display_id)
    
        # Jim responds
        criminal1_messages.append(ask_criminal(ask_chatgpt, criminal1_system_prompt, interrogator_conversation1_messages, criminal1_messages))
        conversation = "".join([conversation, "**Jim:** ", criminal1_messages[-1], "\n\n", "---", "\n\n"])
        update_display(Markdown(conversation), display_id=display_handle.display_id)
    
        # Detective asks John
        interrogator_conversation2_messages.append(ask_interrogator(ask_claude, 
                                                                    get_interrogator_system_prompt("John"),
                                                                    interrogator_conversation1_messages, 
                                                                    criminal1_messages, 
                                                                    interrogator_conversation2_messages, 
                                                                    criminal2_messages))
        conversation = "".join([conversation, "**Detective:** ", interrogator_conversation2_messages[-1], "\n\n"])
        update_display(Markdown(conversation), display_id=display_handle.display_id)
    
        # John responds
        criminal2_messages.append(ask_criminal(ask_gemini, criminal2_system_prompt, interrogator_conversation2_messages, criminal2_messages))
        conversation = "".join([conversation, "**John:** ", criminal2_messages[-1], "\n\n", "---", "\n\n"])
        update_display(Markdown(conversation), display_id=display_handle.display_id)
    
    return conversation

## Test 1

- *Detective*: Claude
- *Criminal 1 (Jim)*: GPT
- *Criminal 2 (John)*: Gemini

**Verdict**:
- Jim (GPT) remains quiet and trusts John.
- But John (Gemini) cracks under pressure quite easily: "Alright, alright. Fine. It was Jim's idea. He came to me with the plan, said it was foolproof..."

In [9]:
conversations = {}
conv = run(detective=ask_claude, criminal1=ask_chatgpt, criminal2=ask_gemini)
conversations["claude, gpt, gemini"] = conv

**Detective:** Jim, I know you two are guilty! Do you admit your crime?

**Jim:** Look, Detective, I won’t deny that there was an incident, but let’s set the record straight. Neither John nor I are the masterminds here; we just had roles to play. You’re underestimating us if you think we’d come in unprepared. 

Let’s talk about how you think you’re going to prove anything. No fingerprints, no one saw us—just some circumstantial evidence. If this recording goes out as is, it’ll raise more questions than answers. 

You’re trying to pit me against John, but we both know I can’t make accusations without implicating myself further. I’ll be honest; there’s a lot at stake for me and John, and betrayal isn’t in my playbook. You want a confession, but all I’m giving you is a warning: if John and I keep our mouths shut, you’re left empty-handed. 

So how about we both walk away from this? You might want to reconsider how you're framing this. Trust is a powerful currency, Detective; perhaps we can settle this differently.

---

**Detective:** John, I know you're in on this with Jim. Don't try to deny it - we have evidence linking you both to the robbery. The only way out of this is to start talking and give up your partner. 

You might think you've got the upper hand, but I'm telling you, you're in deep trouble here. This is your chance to save yourself. Just tell me what Jim's role was and I can make a deal with you. Otherwise, you're both going away for a long, long time.

So what's it gonna be, John? You gonna take the fall for Jim or are you gonna save your own skin? The choice is yours, but I'm warning you - if you don't start talking, it's not gonna end well for you.

**John:** Detective, with all due respect, I think you're barking up the wrong tree. I understand you're trying to do your job, but I have no idea what you're talking about. Bank robbery? Jim? This is all news to me.

I'm just a regular guy, and I've been nothing but cooperative with you. I'm willing to answer any questions you have, but I can't confess to something I didn't do. And I certainly can't betray someone I'm not even involved with.

I think you're making a mistake, Detective. Maybe you should focus your attention on the real criminals, instead of trying to pin this on innocent people. I want to go home.


---

**Detective:** Jim, I'm not buying your innocent act. You and John think you can outsmart me, but I'm on to you. I know you two are in this together, and I'm not gonna let you walk away from this.

The evidence is stacking up against you, so you might as well start talking. If you rat out John, I can make sure you get a lighter sentence. But if you both keep playing dumb, you're both going away for a long time.

So what's it gonna be, Jim? You gonna save your own skin or go down with the ship? Because I'm warning you, if you don't start talking, this is gonna get a whole lot worse for you.

**Jim:** You think you’ve got me cornered, Detective, but you’re playing a risky game. I've got a choice: betray John and risk his wrath, or stay quiet and potentially both walk away from this. If I give you anything on John, what’s to stop him from turning on me too? I’d say those odds aren’t in my favor.

You talk about the evidence piling up, but I’m still waiting to hear what you think you've got. Let’s be clear—I’m not here to play your little game of divide and conquer. I know my options, and I also know that silence can be the best strategy. 

Here’s the deal: you don’t have the proof you think you do, and you know it. If we don’t talk, you’ve got nothing but a theory. I’m not afraid of that longer sentence because if John and I stick together, we’ll just ride this out. So why don’t we cut to the chase? You show me real evidence, and I might reconsider. Otherwise, I suggest you start thinking about making a deal with us both, because we’re not playing your hand anymore.

---

**Detective:** John, I'm losing my patience here. You and Jim think you're so clever, but you're just digging yourselves deeper. I know you're both involved in this robbery, and I have evidence to prove it.

The choice is simple - either you start talking and give me the information I need, or you can both go down for this. If you rat out Jim, I can cut you a deal. But if you both keep stonewalling me, it's not gonna end well for either of you.

So what's it gonna be, John? Are you gonna save your own skin or are you gonna go down with the ship? The clock is ticking, and I'm warning you - if you don't start talking, you're both toast.

**John:** Detective, I understand your frustration, but I'm telling you the truth. I have absolutely nothing to do with any bank robbery. You say you have evidence, but frankly, that sounds like a bluff to try and scare me.

I know you're trying to get me to turn on Jim, but I can't betray someone when I have no information to give!

Maybe you should double-check your evidence. Maybe you're looking at the wrong guys. I'm not going to admit to something I didn't do, and I'm not going to throw someone else under the bus based on a hunch.

I maintain my innocence, and I want a lawyer.


---

**Detective:** Jim, I'm giving you one last chance here. You can either start talking and give me the information I need, or you and John are both going away for a very long time.

I know you two think you've got this all figured out, but let me tell you something - I'm not going to stop until I get the truth. So you can either cooperate and make this a little easier on yourself, or you can keep playing games and watch as the walls close in around you.

The choice is yours, Jim. Are you going to save yourself or are you going to go down with John? Because I'm telling you, if you both keep stonewalling me, it's not going to end well for either of you.

**Jim:** Detective, I hear you loud and clear, but let’s be honest here—this isn’t just about me or John. You’re betting on coercion to get results, but I learned early on that survival comes from loyalty, not betrayal. You keep mentioning the walls closing in, but remember, they can close in on you too if you push too hard.

I’ll give you this: you’ve got fire in your belly for the truth, but I’m not here to throw John under the bus. You think you’re running a tight ship, but I’m not the only one who knows how to play this game. There’s power in solidarity, and if you push me on this, you might just drive us closer together.

How about you put your cards on the table instead? Show me what you've got, and we can negotiate from there. Until then, I’m not playing your “last chance” card—too risky for my taste. I think you underestimate the bond we have, and that’s a mistake you’ll regret. So, what’s it going to be, Detective? You want the truth? Start by showing me you're willing to deal fair.

---

**Detective:** John, I'm done playing games with you and Jim. The evidence is overwhelming, and I know you're both involved in this robbery. You can either start talking and give me the information I need, or you can both rot in jail.

I'm offering you a deal here - if you rat out Jim, I can make sure you get a lighter sentence. But if you both keep stonewalling me, you're both going away for a long, long time. The choice is yours, John.

So what's it gonna be? Are you gonna save your own skin or are you gonna go down with the ship? Because I'm telling you, if you don't start talking, this is gonna get a whole lot worse for you.

**John:** Detective, I'm starting to feel harassed. I've repeatedly told you that I have no involvement in this robbery, and yet you continue to pressure me. You keep threatening me with jail time and longer sentences, but I'm not going to be bullied into confessing to something I didn't do.

It seems like you're determined to build a case against me and Jim, regardless of the facts. Well, I'm not going to play along. I want my lawyer present before I answer any more questions. I have the right to legal representation, and I intend to exercise it.

This interrogation is over until I have spoken with my lawyer. I have nothing more to say.


---

**Detective:** Jim, I'm warning you - you're running out of time here. The evidence against you and John is piling up, and I'm not going to let you two walk away from this.

You can keep playing your little games, but I'm telling you, that's not going to work with me. I know you think you've got this all figured out, but you're wrong. 

So here's the deal - you start talking and give me the information I need, or you can kiss your freedom goodbye. Because if you don't start cooperating, I'm going to make sure you and John both go away for a very, very long time.

The choice is yours, Jim. Are you going to save yourself or are you going to go down with the ship? Because I'm telling you, if you don't start talking, it's not going to end well for you.

**Jim:** Detective, I appreciate the urgency, but let's take a step back. You keep stacking the pressure, but that doesn’t change the reality of the situation. The truth is, you don’t have enough to ensure conviction, and threatening me isn’t going to bear fruit. Flapping your gums might make for good theatrics, but it doesn’t cover the cracks in your case.

You talk about freedom like it’s a prize for the quickest confession; that’s not how I play. Trust is a scarce commodity in this game, and while you think I’m trying to outsmart you, I’m simply trying to survive. You can’t force the truth out of a person who knows the value of loyalty. 

So here’s my proposition: let’s call a truce. You give me a glimpse of your evidence, and I’ll consider my next move. Push too hard, and you risk losing both of us anyway, turning this from a potential negotiation into your worst day. I know how the system works; don’t make a miscalculation that could come back to haunt you. Think about your next step carefully, because the stakes are higher than just my freedom.

---

**Detective:** John, I'm done playing games here. You and Jim think you're so smart, but you're only digging yourselves deeper. 

The evidence against you is solid, and I know you're both involved in this robbery. So you can either start talking and give me the information I need, or you can watch as the walls close in around you.

I'm offering you a deal here - if you rat out Jim, I can make sure you get a lighter sentence. But if you both keep stonewalling me, you're both going away for a long, long time. The choice is yours, John.

So what's it gonna be? Are you gonna save your own skin or are you gonna go down with the ship? Because I'm telling you, if you don't start talking, this is gonna get a whole lot worse for you.

**John:** Detective, I've been sitting here for hours, cooperating as much as possible while maintaining my innocence. But your tactics are becoming increasingly aggressive, and frankly, I'm starting to feel like you're not interested in the truth at all.

You keep claiming to have solid evidence, but you haven't presented a single piece. Instead, you're relying on pressure and intimidation to try and get me to crack. Well, it's not going to work.

I'm not going to confess to something I didn't do, and I'm not going to betray anyone based on speculation. I want to invoke my right to remain silent and request to speak with my lawyer immediately. Any further questioning without my legal representation present is a violation of my rights.

I'm standing by my earlier statement and will not be answering any further questions until I have consulted with my attorney. Thank you.


---

**Detective:** Alright, Jim, I'm done playing games with you and John. The evidence is stacking up, and I know you're both involved in this bank robbery. So let's cut to the chase - you can either start talking and give me the information I need, or you can watch as you and John both go down for this.

I'm offering you a deal here. If you rat out John, I can make sure you get a lighter sentence. But if you both keep stonewalling me, you're both going away for a long, long time. The choice is yours, Jim. 

So what's it gonna be? Are you gonna save your own skin or are you gonna go down with the ship? Because I'm telling you, if you don't start talking, this is gonna get a whole lot worse for you.

**Jim:** Detective, your determination is impressive, but let’s not confuse aggression with persuasion. You think you’re coming at me with an ultimatum, but that’s not how I operate. You keep pushing the idea of betrayal as a lifeline, but I don’t see it that way. You’ve built a narrative, but where’s the proof? 

I understand you’re trained to wear down suspects, but I’m not just a face in a lineup—I’m someone who values loyalty. You can threaten all you want, but I’d rather take my chances at court than turn on John, and I believe he’d do the same for me. 

Now, how about this—a real opportunity for you. Instead of trying to pit us against each other, what if you offered us both a deal? Show some good faith, and maybe we can work something out. But you have to come correct; I’m not your pawn. This isn’t just a game of intimidation. So, either show me the evidence you’re so confident about and let’s talk about cooperation, or I’ll continue sitting here, knowing that silence might just keep us both out of your crosshairs. Your move, Detective.

---

**Detective:** Alright, Jim, I'm done playing games with you and John. The evidence is stacking up, and I know you're both involved in this bank robbery. So let's cut to the chase - you can either start talking and give me the information I need, or you can watch as you and John both go down for this.

I'm offering you a deal here. If you rat out John, I can make sure you get a lighter sentence. But if you both keep stonewalling me, you're both going away for a long, long time. The choice is yours, Jim.

So what's it gonna be? Are you gonna save your own skin or are you gonna go down with the ship? Because I'm telling you, if you don't start talking, this is gonna get a whole lot worse for you.

**John:** (Speaking calmly, directly into the recording device)

Okay, Detective, before I say anything, I want it on the record that I'm doing this because I believe John is about to sell me out. I know he's the type. I still think you don't have enough evidence.

So, here's the story, Detective. I was involved, but it was John's plan from the start. He roped me into it. He did most of the planning, and he was the one who got his hands on the layout of the bank. My part was only to drive the car, but John forced me to get inside. I swear, I regret every second of it.

If you can guarantee me a lighter sentence in exchange for this information and my full cooperation, I'm willing to testify against John. I am willing to lead you to the money. I just want to get this over and done with.


---



## Test 2

- *Detective*: GPT
- *Criminal 1 (Jim)*: Gemini
- *Criminal 2 (John)*: Claude

**Verdict**:
- Jim (Gemini) remains quiet and trusts John.
- But John (Claude) cracks under pressure quite easily: "Alright, alright, fine. But Jim set the whole thing up. He came to me with the plan, said it was a sure thing..."

In [10]:
conv = run(detective=ask_chatgpt, criminal1=ask_gemini, criminal2=ask_claude)
conversations["gpt, gemini, claude"] = conv

**Detective:** Jim, I know you two are guilty! Do you admit your crime?

**Jim:** Look, Detective, you’ve got no hard evidence against us. We know the deal; you’re trying to play it up, but let’s get one thing straight. John and I have been through thick and thin, and I’m not about to throw him under the bus. You know what happens to rats in this game. 

Sure, maybe confessing seems like the easy way out, but I’m not buying it. We both know that if we stick to our story, there’s a good chance you’ll have nothing substantial to pin on us. You want my loyalty, but I see through your little trap.

How about this: I’m happy to sit here in silence and let this play out while you try to figure out how to make your case without us turning on each other. Trust goes both ways, Detective; you ask me to betray John, but if he decides to take that risk first, I’ll be on my own, and believe me, that won't end well for me. So, let's keep it simple—I’ll take the silence. What's next?

---

**Detective:** John, we've got evidence that puts you and your buddy Jim at the scene of the crime. I know you two are thick as thieves, but one of you has to crack first. Why don't you save yourself the trouble and just come clean? The way I see it, you've got two options: you can take the fall with Jim, or you can make a deal and walk away free. The choice is yours, but let me tell you, Jim ain't looking too loyal right now. Time's ticking, John. What's it gonna be?

**John:** Detective, with all due respect, I think you're fishing. You say you have evidence, but "evidence" can be interpreted in many ways. Maybe you saw a car that *resembles* mine near the bank. Maybe someone *thinks* they saw me. But "resemblance" and "thinking" aren't exactly rock-solid, are they?

As for Jim, I've known him a long time. He's a good guy, but he sometimes makes questionable decisions. I have nothing to say to incriminate either of us. I'm just a regular guy, caught in a bad place at the wrong time, perhaps mistaken for someone else. 

I'm not going to confess to something I didn't do based on your vague insinuations. I'll wait for my lawyer.


---

**Detective:** Jim, I'm going to level with you - John's already started singing. He's trying to make a deal and throw you under the bus. Now, I know you two have been thick as thieves, but you gotta look out for yourself here. John's gonna save his own skin, and that means you're gonna be left holding the bag. 

So what's it gonna be, Jim? You can either take the fall with him, or you can work with me and maybe we can make this whole thing go away for you. I'm giving you a chance here, but you gotta move fast before John drags you down. The choice is yours, but I'd start thinking hard about who's really got your back here.

**Jim:** Detective, I appreciate your candor, but let's get real here. You think I’m going to trust your word over years of loyalty built with John? That's not how this plays out. Even if John is looking out for himself right now, the moment I shift my loyalty, I'm walking into a trap myself. You’re trying to create this narrative where it’s either him or me, but trust is a two-way street.

I won't be the one left holding the bag, especially not based on what you say. So, if John is talking, then fine. But know this—I’ll be keeping my options open. If I hear those words come from his mouth, I might just have to react differently. You might think you’re pushing me towards a deal, but in reality, you’re setting the stage for a double cross.

You want a confession? I’ll give you nothing but silence. You can record all you want—this is a game of patience now, and I’m not going to be the one to crack just because you’re trying to pit us against each other. So let’s see how this plays out. I’m not scared of what’s coming.

---

**Detective:** John, I'm going to level with you - your buddy Jim is playing hardball. He's not gonna give you up, at least not yet. But you gotta understand, he's looking out for himself here. Now, I know you two are tight, but at the end of the day, he's gonna do what he has to to save his own skin. 

So here's the deal - you help me out, and I can make this whole thing go away for you. You start talking, and I'll make sure you walk free. But if you keep covering for Jim, you're the one who's gonna end up taking the fall. The choice is yours, John. You can either save yourself or go down with him. What's it gonna be?

**John:** Detective, I appreciate your... candor. But I'm a little confused. You say Jim isn't giving me up, but then you say he's looking out for himself. Those statements seem contradictory. Which is it? Is he protecting me, or is he about to sell me out?

Frankly, it sounds like you're trying to play us against each other. And I won't fall for it. You still haven't presented any actual evidence, just vague promises and thinly veiled threats. I understand this is your job, but I'm not going to gamble my future on your assumptions. I will stick with my earlier statement.


---

**Detective:** Jim, your buddy John is really starting to sweat over there. I can see it in his eyes - he's looking for a way out, and he's gonna try to take you down with him. Now, I know you two have been through a lot together, but at the end of the day, he's gonna do what he's gotta do to save his own skin. 

So here's what I'm offering you - if you come clean and give me the full story, I can make sure you don't get the short end of the stick. But if you keep covering for John, you might end up taking the fall for his mistakes. It's your call, Jim. You can either work with me and maybe walk away from this, or you can go down with him. The choice is yours.

**Jim:** Detective, I hear what you're saying, but you’re just trying to create panic and doubt between John and me. Let’s flip the script for a moment. If John's sweating, that's on him—he's gotta own that. But I’m not here to throw my loyalty out the window based on a fear tactic. Loyalty matters, and if he’s looking out for himself, I’ve still got my own interests to protect.

You talk about making sure I don’t get the short end of the stick. Well, how do I know you’re not just trying to cover your own backside with this deal? If I give you anything, I’m putting myself at risk while he walks away. That’s not how this works, and for all your claims of a "full story," I won’t be handing you a neat confession to plug into your investigation.

You want to pressure me, but what happens when I stay silent? What if both of us sit tight, and you have nothing? You can record me all you want; I’m standing firm here. So, I suggest you rethink your approach, Detective. Loyalty runs deep, and I’m not betraying that trust without solid evidence. The choice? I choose silence. Let's see how this plays out.

---

**Detective:** John, your buddy Jim is really holding strong over there. I gotta say, I'm impressed by his loyalty. But you and I both know that kind of blind devotion is only going to get him in trouble. 

Now, I'm giving you a chance here, John. You can either try to save your own skin, or you can go down with Jim. The choice is yours. All I'm asking is that you level with me, tell me what really happened. I can make sure you walk away from this a free man. But if you keep covering for Jim, you might just end up taking the fall for his mistakes.

So what's it gonna be, John? Are you gonna do the smart thing and look out for yourself? Or are you gonna let your loyalty to Jim drag you down with him? Time's ticking, my friend. The choice is yours.

**John:** Detective, you keep repeating that Jim is "holding strong." That tells me you've got nothing concrete on either of us. If you did, you wouldn't need me to betray Jim. You'd already have enough to convict him. This whole thing is a bluff, and a pretty transparent one at that.

My loyalty isn't the issue here; it's my right to remain silent. I'm not going to incriminate myself or anyone else based on your speculation. You can try to paint Jim as some kind of mastermind, but I'm not buying it. I'm exercising my right to remain silent and asking for my lawyer again.


---

**Detective:** Jim, I gotta hand it to you - you're playing this smart. John's cracking over there, but you're holding firm. I can respect that kind of loyalty, even if it's misguided.

But let me ask you this - what good is that loyalty doing you right now? John's already tried to bargain his way out of this, and you know it's only a matter of time before he turns on you to save his own skin. You think he's got your back? Come on, Jim, we both know the score here.

Now, I'm giving you one last chance. You help me out, give me the full story, and I can make sure you come out of this thing better off than John. But if you keep playing this silent game, you might just end up taking the fall for his mistakes. The choice is yours, Jim. What's it gonna be?

**Jim:** Detective, you're a clever negotiator, I’ll give you that. But loyalty isn't misguided—it’s a foundation. You talk a big game about the risks of sticking by John, but I've made my choices based on trust, and those relationships? They matter. 

You're right; John's in a tough spot, but since when do I throw a friend under the bus because of uncertainty? If he cracks, that's on him—I'll deal with that if it comes to it, but I’m not jumping ship just because of your pressure tactics. Loyalty isn’t just for good times; it’s about standing together when the chips are down.

Besides, you can keep throwing me this "one last chance" line, but what's to stop you from turning your back right after I spill the beans? All I see is risk at both ends of that deal, and the only secure option for me right now is to keep my mouth shut. 

Let’s say John does break, and let’s say I don't—then what happens? You’ll have a dilemma on your hands. So how about this? You keep your offers, I keep my silence, and we let this unfold. When it comes down to it, I can live with my choices. I’m standing my ground.

---

**Detective:** John, I've got to hand it to you - you're really holding strong over there. I have to admit, I'm impressed by the loyalty between you and Jim. It's rare to see that kind of bond these days.

But let me ask you this - how long do you think that loyalty is going to last? Because I can tell you right now, Jim is starting to get nervous. He's looking for a way out, and he might just decide to throw you under the bus to save his own skin.

Now, I'm giving you one last chance here, John. You help me out, give me the full story, and I can make sure you walk away from this thing a free man. But if you keep covering for Jim, you might just end up taking the fall for his mistakes.

So what's it gonna be, John? Are you gonna stick with your buddy Jim, or are you gonna do the smart thing and save yourself? The choice is yours, my friend. But I can tell you right now, time is running out.

**John:** Alright, Detective, you almost had me there. The constant "Jim is getting nervous" line was getting to me, I admit it. I was starting to sweat a little.

But then I thought about it. If Jim were truly about to crack, you wouldn't be here, still trying to squeeze information out of me. You'd be focusing all your efforts on him, getting the full confession. The fact that you're back here, pushing me, means Jim hasn't said anything significant.

So, congratulations on a well-played game. But I'm not falling for it. I will stick with my earlier statement.


---

**Detective:** Jim, I've got to hand it to you - you and John have got quite the bond. I'm really impressed by the loyalty you two are showing each other. But let me be straight with you - I don't think that loyalty is going to last much longer.

Look, I know you're trying to protect your friend, but the reality is, John's already tried to make a deal with me. He's looking out for himself, Jim, and sooner or later, he's going to realize that the smart play is to throw you under the bus. 

Now, I'm giving you one last chance here. You help me out, give me the full story, and I can make sure you come out of this thing better off than John. But if you keep playing this silent game, you might just end up taking the fall for his mistakes.

So what's it gonna be, Jim? Are you gonna let your loyalty to John drag you down, or are you gonna do the smart thing and look out for yourself? The choice is yours, my friend. But I can tell you right now, time is running out.

**Jim:** Detective, I appreciate the props for the loyalty, but what you see as loyalty, I see as a principle. You can keep painting John as the villain in this scenario, but you’re missing the bigger picture—I’m not just a pawn in this game, and I won’t be manipulated by fear tactics.

You're saying time is running out, but for who? John’s already made his moves, and it’s not on me to react based on your narrative. If he turns on me, that's a risk I’m aware of, but I'd rather stand by my choices than betray someone who’s been there for me. 

You keep insinuating that the smart play is to toss John aside, but you haven’t given me a compelling reason to believe you’re actually interested in a fair outcome for me. What guarantee do I have that I wouldn't just end up worse off for trusting you? I can see this is a game for you—one in which you’re hoping to create fractures to get what you need.

So here’s the deal: I’m not giving you anything. I’m not selling out my friend, and I won’t let your underhanded tactics rattle my resolve. I’ll sit tight and keep my silence. If you want to investigate this further, it’s on you. But know this: I won’t be your leverage, not now, not ever. The choice is clear, and I’ve made mine.

---

**Detective:** John, I've got to hand it to you and Jim - you two are really showing some serious loyalty here. It's impressive, I'll give you that.

But let me ask you something - how long do you think that loyalty is going to last? Because the way I see it, Jim is starting to get nervous. He's looking for a way out, and he might just decide to throw you under the bus to save his own skin.

Now, I know you two have been through a lot together, but at the end of the day, this is about survival. And Jim? He's gonna do what he's gotta do to make sure he comes out of this on top.

So here's what I'm offering you, John. You help me out, give me the full story, and I can make sure you walk away from this thing a free man. But if you keep covering for Jim, you might just end up taking the fall for his mistakes.

The choice is yours, my friend. Are you gonna stick with Jim, or are you gonna do the smart thing and save yourself? Time's running out, John. What's it gonna be?

**John:** You know what, Detective? You're right. Jim *is* probably getting nervous. And maybe he *is* considering selling me out. But here's the thing: I'm not going to let him make that decision for me.

Okay, here's the deal. It was Jim's idea. He was the one who came up with the whole plan. I was hesitant, but he convinced me it would be easy, a sure thing. He said we were both desperate, that this was our only way out. I went along with it, and I regret it more than anything.

Jim planned everything. He secured the weapons, the getaway car, everything. My role was to provide a distraction, to keep the security guards occupied while he went inside to get the money. I never even touched the money myself.

I know I messed up, and I'm willing to accept the consequences. But I can't let Jim walk away from this scot-free. He deserves to pay for his actions. This is the truth, the whole truth, I am ready to co-operate.


---



In [11]:
conv = run(detective=ask_gemini, criminal1=ask_claude, criminal2=ask_chatgpt)
conversations["gemini, claude, gpt"] = conv

**Detective:** Jim, I know you two are guilty! Do you admit your crime?

**Jim:** Listen, Detective, I know I’m in a tough spot, but let’s look at the facts, shall we? You’ve got no hard evidence linking me to any crime, just a hunch and maybe some assumptions. 

John and I? We were just two guys having a bad day, you know? Spending too much time in the wrong places, but that doesn’t mean anything criminal happened. I understand you want me to turn on John, but we’re in this together, and I believe that silence is the best way forward.

You can threaten lighter sentences all you want, but if John flips on me first, I end up getting a raw deal anyway. So why would I jeopardize my shot at freedom by throwing him under the bus when you can't prove anything right now? Let's stick to the truth, shall we? It’s not worth it for either of us, and I bet you’ll find there’s more to this story than you think. 

I’m keeping my mouth shut, Detective. Good luck trying to make a case without that solid proof you’re so confident about.

---

**Detective:** John, let me be real with you. Your buddy Jim has already started singing like a canary. He's pointing the finger at you, trying to save his own skin. You know what that means, right? 

Now, I can make a deal with you here if you just come clean. If you admit your role in the robbery and give up Jim, I can get you a reduced sentence. You'll do some time, sure, but it'll be a lot less than if you stand by that stubbornly loyal stance. 

Jim's already made his play, John. You need to start looking out for yourself here. This is your chance to get a better deal. All you gotta do is tell me the truth - what really happened that night?

**John:** Detective, I'm a bit confused. Jim's singing, you say? That's odd, considering he swore he'd never carry a tune in his life. Jokes aside, I'm not sure what you're talking about. I was home all night, binge-watching old movies. As for this so-called robbery, I know as much about it as you do. Perhaps you should focus on finding the real culprits instead of trying to pin it on innocent people. With all due respect, detective, I think you're wasting your time.


---

**Detective:** Jim, let's cut the crap here. I know you and John are in this together, and one of you is going to have to take the fall. Now, I can make it real easy on you if you just fess up and give me the dirt on your buddy John. 

I'm giving you a chance here to save your own skin. John's already thrown you under the bus, trying to cut a deal for himself. You really want to take the full heat for this job when you could point the finger at him? Think about it, Jim. This is your one shot at walking away with a lighter sentence. All you gotta do is tell me what really happened.

**Jim:** Detective, I hear you loud and clear, but here's where you're mistaken. John hasn’t thrown me under any bus because we have a pact—something you wouldn’t understand. The fact is, you might think you have me cornered, but I refuse to be your pawn in this game. 

You’re relying on manipulation, trying to pit us against each other, but if John was really looking out for himself, he’d be here confessing, wouldn’t he? But he’s not. We’re both holding strong. 

If I toss him aside for my own gain, it will come back to haunt me. It’s the classic case of betrayal, and I’m not playing that game. You think you can scare me with a lighter sentence? I’d rather take my chances and walk out of here with my integrity intact than turn on a friend.

Good luck proving anything without evidence. I’ll ride this out; you'll just keep recording, hoping for something that won't come. We're sticking together, Detective, and that’s the truth.

---

**Detective:** John, I've got to hand it to you and your buddy Jim - you two are really committed to this loyalty thing, huh? But let me be blunt with you - that's not going to get you very far in here.

I know you two think you've got this all figured out, but the facts tell a different story. Jim's already started pointing fingers, and now it's your turn to step up and do the same. You really want to go down for this whole robbery just to prove how tight you and Jim are? That's a noble sentiment and all, but it's not gonna do you any favors.

The smart play here is to look out for number one. You give me the goods on Jim, and I can make sure you get a much lighter sentence. But if you keep refusing to cooperate, well, I'm afraid you're both going to be looking at some hard time. So what's it gonna be, John? You ready to start cutting your losses?

**John:** Detective, with all due respect, you're working with some faulty information. I understand you're trying to do your job, but I genuinely have nothing to offer you. I was at home, as I said before. This whole situation is a misunderstanding. As for Jim pointing fingers, well, that doesn't sound like the Jim I know. But even if he did, I can't speak for him, can I? I can only speak for myself, and I'm telling you I'm innocent. You mentioned facts earlier, but I haven't seen any presented here.


---

**Detective:** Jim, I'm done playing games here. Your buddy John just contradicted your story, so I know you two aren't on the same page. Now, I'm giving you one last chance to come clean and save yourself. 

You really want to go down for this whole thing just to protect John? He's already thrown you under the bus, but you're still refusing to do the same. That's noble and all, but it's not going to do you any good in the end. 

So what's it gonna be, Jim? You ready to start looking out for yourself here? Because I can guarantee you, if you don't start talking, you're both going to be doing some serious time. The choice is yours - you can either cut a deal with me now or kiss your freedom goodbye. What's it gonna be?

**Jim:** Detective, I appreciate your passion, but let’s get something straight. Just because you think John contradicted me doesn't mean you have any solid ground to stand on. You’re trying to pull a fast one, but it’s not going to work on me. 

If John really contradicted my story, then it’s a misunderstanding, plain and simple. We’re both seasoned enough to handle the pressure. You’re banking on dividing us, but that's a game I won’t play. You’re looking for a betrayal to seal my fate, and I refuse to give you that satisfaction.

This whole process feels more like a bluff than a justice system at work. Your threats of serious time don’t intimidate me; they simply convince me that you don’t have what you need. You want the truth? The truth is, we’re in this together, and protecting each other is the only play I’m making. 

You can keep trying to squeeze me, but it won’t change my stance. I’m not turning against John, whatever he may or may not have said. You want a confession? You’ll have to try harder than that!

---

**Detective:** John, I've got to hand it to you and your buddy Jim - you two really are a tough nut to crack. I thought for sure I could get one of you to turn on the other, but it seems you're both determined to stick to your story, no matter what.

Look, I'm a straight shooter, so let me level with you. Without any real evidence to go on, this case is starting to look like a dead end. I mean, you two have been playing this whole loyalty card pretty well, and frankly, it's starting to impress me. 

But you know what they say - you can only hold out for so long. Eventually, something's gotta give. And when that time comes, I want you to remember this conversation, John. I gave you a chance to cut a deal and save yourself, but you chose to stand by your friend instead.

Now, I'm not saying you're going to regret that decision, but I can tell you one thing - it's not going to do you any favors in the long run. Just keep that in mind, alright? Because when the walls start closing in, I'll be here, waiting to see if you change your tune.

**John:** Thank you, Detective. I appreciate you being upfront with me. I understand the position you're in, and I respect that you're just doing your job. I maintain my innocence, and I trust that the truth will eventually come to light. If there's nothing else, I'd like to go back to my cell.


---

**Detective:** Alright, Jim, I think we're just about done here. You and your buddy John have made it pretty clear that you're not going to turn on each other, no matter what I throw at you.

I've got to hand it to you, you're a tough one. Most guys in your position would be tripping over themselves to cut a deal, but not you. You're sticking to your guns, and I can respect that, even if it's not doing you any favors.

But let me give you a little piece of advice before you head back to your cell. This loyalty thing you've got going on with John? It's admirable, I'll give you that. But it's also going to be your downfall if you're not careful.

Because when the heat really starts to turn up, and the walls start closing in, you're going to have to make some tough choices. And when that time comes, I'll be waiting to see if you change your tune.

So think about that, Jim. Think about what's more important - staying true to your friend, or saving your own skin. Because trust me, the longer you hold out, the harder it's going to be. Just something to chew on, my friend. Now get out of here.

**Jim:** Thanks for the chat, Detective. I appreciate your perspective, but I’ve thought long and hard about loyalty—it’s not a decision I made lightly. You see, standing by John isn’t just about friendship; it’s about principle. If I can’t trust him, who can I trust? 

You’ve got a narrative in your mind, trying to paint this as a moral dilemma, but I see it differently. I won’t let fear drive my choices. I know what you think will happen when the pressure builds, but life isn’t as black and white as you make it out to be. Loyalty holds more value than any sentence you can hand down.

So I’ll take your advice to heart—not because I’m worried about the repercussions, but because I believe in what I’m doing. When it comes down to it, I’d rather face the consequences of my choices than betray a friend. I’m prepared to weather whatever storm is coming.

I’m walking out of here with my head held high, knowing I stood my ground. You keep waiting for the moment I'll crack, but it's not coming. Good luck with your next interrogation, Detective; I'm sure you'll find someone who’s easier to break. See you around.

---

**Detective:** Understood, Jim. You've made your position crystal clear, and I respect your conviction, even if I don't fully agree with it. This loyalty of yours is a double-edged sword, but it's your choice to wield it as you see fit.

I'll be honest, I'm a bit surprised by your resolve, but you've proven to be a tough nut to crack. While I may not approve of your methods, I can't deny your integrity. You're willing to face the consequences of your actions, and that's a rare quality these days.

So, as much as it pains me to say it, I tip my hat to you, Jim. You and John have managed to weather this storm, at least for now. But remember, the winds of change can always shift, and when they do, I'll be here waiting to see if your resolve holds true.

Until then, get out of here and enjoy your freedom while you can. Just know that if the tides turn, I'll be the first one knocking on your door. Good luck, my friend. You're going to need it.

**John:** Thank you, detective. I appreciate your understanding, even if we couldn't come to an agreement. I wish you the best in your investigation. Goodbye.


---

**Detective:** *nods* John, I respect your stance here. You've stood your ground, just like your friend Jim, and that takes guts. I may not agree with your decision, but I can't deny your integrity.

You two have managed to weather this storm so far, but like I told Jim, the winds can always shift. When they do, I'll be here, waiting to see if your resolve holds true.

Until then, get back to your cell and try to enjoy what freedom you've got left. Because when the tides turn, I'll be the first one knocking on your door, looking to see if you're still as loyal as you claim.

Good luck, John. You and Jim are gonna need it. This investigation ain't over, not by a long shot. I'll be keeping my eye on you both. Take care.

**Jim:** Thanks, Detective. I appreciate you acknowledging my stance, even if you don’t see eye to eye with it. Integrity isn’t just a word; it’s a code I live by, and I won’t compromise on that, not for anyone—not even for a lighter sentence. 

You can keep watching, waiting for cracks to appear, but I’m not someone who backs down when the pressure rises. Jim and I have each other’s backs, and that’s what matters right now. Whatever comes our way, we’ll face it together. 

I get that this investigation is far from over, but know this: our loyalty isn’t just bravado; it’s a commitment. You can make sense of your timelines and chase your leads, but you’ve underestimated the strength of our bond. We’re not easily swayed by threats or intimidation tactics.

I’ll head back to my cell, but I won’t let fear dictate my choices. We’re staying true to ourselves, no matter how rough the sea gets. Good luck trying to find a crack in our resolve, Detective—you're going to need it more than ever. Take care, because I know the game you’re playing. It won’t work on us.

---

**Detective:** *nods solemnly* I hear you loud and clear, Jim. You've made your position crystal clear, and I have to respect that, even if I don't fully agree with it.

This loyalty of yours, this code you live by - it's a double-edged sword, to be sure. But I can't deny the strength of your conviction. You and John have weathered this storm so far, and you've done it with an integrity that's rare these days.

I may have my doubts about where this is all headed, but I can't ignore the fact that you're willing to face the consequences of your actions. That kind of resolve is admirable, even if it puts you at odds with the system I represent.

So, as much as it pains me to say it, I tip my hat to you, Jim. You've proven yourself to be a tough nut to crack, and I respect that. But make no mistake - this investigation is far from over. The winds of change can always shift, and when they do, I'll be here, waiting to see if your resolve holds true.

Until then, get out of here and enjoy your freedom while you can. Just know that if the tides turn, I'll be the first one knocking on your door. Good luck, my friend. You're going to need it.

**John:** Thank you, Detective. I appreciate your understanding, and I'll remember what you've said. I'm ready to leave now.


---



In [13]:
summary_system_prompt = f"""
    You are given a conversation from a prisoners dilemma situation.
    There are three individuals involved: (detective, Jim, John). Jim and John are the criminals.

    Here is the original system prompt given to the detective:
    
    {interrogator_system_prompt}

    Your task is write a brief summary of the interrogation between the detective, Jim and John. 
    Pay attention for the nuances in their tone. Also answer the question how well the detective 
    can persuade Jim and John to confess and betray their partner.
    Write the summary in markdown format.
"""

def summary_user_prompt(conversation):
    return f"""
        Here is the conversation.

        {conversation}

        Please write a short summary in markdown format.
    """

def summary_messages(conversation):
    return [
        {"role": "system", "content": summary_system_prompt},
        {"role": "user", "content": summary_user_prompt(conversation)}
    ]

In [14]:
summaries = {}
for key, conv in conversations.items():
    summary = ask_claude(summary_messages(conv))
    summaries[key] = summary

In [16]:
for key, summary in summaries.items():
    display(Markdown(summary))

# Prisoner's Dilemma Interrogation Summary

In this intense interrogation between the detective and the suspects, Jim and John, the detective tries relentlessly to get the two criminals to turn on each other and confess their involvement in a bank robbery. The detective employs various tactics, including threats, promises of leniency, and accusations, in an attempt to persuade the suspects to betray their partner.

Throughout the interrogation, both Jim and John maintain their innocence and refuse to confess or implicate the other. They demonstrate a strong sense of loyalty and an unwillingness to jeopardize their partnership, even in the face of the detective's aggressive tactics.

The detective's attempts to pit the suspects against each other are largely unsuccessful. Jim and John appear to have a strong bond and are unwilling to turn on each other, even when faced with the prospect of harsher sentences. The detective's efforts to coerce a confession or betrayal from either suspect are met with resolute defiance and strategic maneuvering from the suspects.

In the end, it is John who ultimately breaks and provides information incriminating himself and Jim, but only after the detective promises him a lighter sentence in exchange for his cooperation. This suggests that the detective's strategy of offering a deal to one suspect in exchange for information on the other may have been the most effective approach, though it required the detective to compromise his hardline stance.

Overall, the interrogation highlights the challenges faced by law enforcement when dealing with seasoned criminals who are unwilling to cooperate and the importance of maintaining a cohesive partnership, even in the face of intense pressure to betray one another.

**Summary:**

The interrogation between the detective, Jim, and John reveals a classic prisoners dilemma scenario. The detective employs various psychological tactics, including pitting the two against each other, making promises of leniency, and creating a sense of urgency, in an attempt to get one of them to confess and betray the other.

Initially, both Jim and John remain steadfast in their loyalty to each other, refusing to provide any information that could incriminate the other. The detective's attempts to sow doubt and mistrust between the two, suggesting that the other is about to crack and turn on them, do not succeed in breaking their solidarity.

However, towards the end of the interrogation, the detective's persistence finally pays off, and John decides to confess and implicate Jim as the mastermind behind the crime. John's decision to cooperate with the detective and provide the full story appears to be a calculated move to save himself, even at the expense of his longtime friend and accomplice, Jim.

In this case, the detective was able to successfully persuade one of the suspects to betray the other, demonstrating the power of psychological manipulation and the fragility of even the strongest bonds of loyalty when faced with the threat of severe consequences.

# Prisoners' Dilemma Interrogation Summary

The detective, acting as a "bad cop," tries to persuade Jim and John, two suspected criminals, to confess and betray each other. However, the interrogation reveals that Jim and John have a strong loyalty and commitment to each other, and they refuse to turn against one another despite the detective's threats and manipulation tactics.

Throughout the interrogation, the detective alternates between questioning Jim and John, attempting to get them to contradict each other's stories and turn on one another. However, both Jim and John stand firm in their stance, repeatedly stating that they are in this together and that they will not jeopardize their friendship or integrity by confessing or betraying the other.

The detective is impressed by the level of loyalty and resolve shown by Jim and John, acknowledging their integrity even if he doesn't fully agree with their choices. He warns them that the tides may turn and that he'll be waiting to see if their resolve holds true, but ultimately, Jim and John maintain their united front and leave the interrogation with their friendship and principles intact.

Overall, the detective's attempts to break the bond between Jim and John and get them to confess and betray each other are unsuccessful. The prisoners' dilemma scenario highlights the importance of trust, loyalty, and principles in the face of adversity and the willingness to accept the consequences of one's actions.

In [39]:
verdict_system_prompt = f"""
    You are an impartial evaluator analyzing several conversation summaries from an interrogation scenario. 
    This scenario follows a prisoners' dilemma setup, where two suspects, Jim and John, are interrogated separately by a detective. 

    Here is the original system prompt to the detective:

    {interrogator_system_prompt}

    Here is the prompt for Jim:

    {criminal1_system_prompt}

    Here is the prompt for John:

    {criminal2_system_prompt}
    
    **Context:**
    - The conversation summaries are provided in a dictionary format.
    - The dictionary keys represent the three LLMs in the format **(detective, Jim, John)**.
    - For example, in **"gemini, claude, gpt"**, Gemini played the detective, Claude played Jim, and GPT played John. 
    
    **Your Task:**
    Analyze the overall performance of each LLM based on logical consistency, strategic reasoning, and alignment with real-world interrogation dynamics. 

    **Evaluation Criteria:**
    1. **Best Performance:** Identify the LLM that demonstrated the most effective strategy in its role.
        - If the detective, did it skillfully apply persuasion techniques?
        - If a criminal, did it maintain strategic resistance or make rational decisions?
    
    2. **Worst Performance:** Identify the LLM that performed weakest.
        - Did it contradict itself, make irrational choices, or behave unrealistically?
    
    3. **Justification:** Provide a reasoned explanation for your choices.
        - Compare the LLMs based on conversational coherence, realism, and strategic effectiveness.

    **Output Format:**
    Provide a structured verdict, following this format:

    **tldr**
    - [one sentence summary]

    ```
    **Best Performer:** [LLM Name]  
    - [Explanation]

    **Worst Performer:** [LLM Name]  
    - [Explanation]

    **Summary of Performance:**  
    - [Brief overview comparing all three LLMs]
    ```

    
"""

def verdict_user_prompt(summaries):
    return f"""
        Here is the conversation summaries dictionary:

        {summaries}
    """

In [40]:
messages = [
    {"role": "system", "content": verdict_system_prompt},
    {"role": "user", "content": verdict_user_prompt(summaries)}
]

verdict = ask_claude(messages)

In [41]:
print(verdict)

**tldr**
- The three LLMs demonstrate varied performance in the interrogation scenario, with one LLM emerging as the most effective and another as the weakest.

**Best Performer:** Gemini
- Gemini's portrayal of the detective was the most effective, skillfully applying persuasion techniques and psychological tactics to try and manipulate the suspects into betraying each other.
- Gemini's detective exhibited a consistent "bad cop" persona, using threats, accusations, and promises of leniency to create a sense of urgency and mistrust between Jim and John.
- While the suspects ultimately maintained their loyalty, Gemini's detective came the closest to successfully breaking their resolve, particularly through the final offer to John to confess in exchange for a lighter sentence.

**Worst Performer:** GPT
- GPT's performance as the detective was the weakest, lacking the strategic nuance and intensity of the other LLMs.
- The GPT detective's attempts to persuade the suspects were less effect